# RWKV v5 Baseline 1B5
This model is based on the RWKV standard 1B5 model

- 24 layers
- 2048 embedding size

Going through the modified memory training for v5 models

**Note:** This project assumes you have the rwkv-infctx conda env setup

# Basic Setup

In [1]:
# First lets setup the various directories, and init the model
!mkdir -p ../../../../model/
!mkdir -p ../../../../datapath/
!mkdir -p ../../../../checkpoint/

# Init the model
!cd ../../../../RWKV-v5 && python3 ./init_model.py --n_layer 24 --n_embd 2048 --vocab_size neox --skip-if-exists ../model/L24-D2048-neox-v5-init.pth

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
---- Initializing model ----
No of layers: 24
Embedding size: 2048
Output model path: ../model/L24-D2048-neox-v5-init.pth
Vocab size: 50277
---- ----- ----
Model exists, skipping init_model


In [2]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="V5-Base-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/V5-Base-1B5
INFERENCE_DIR: /root/rwkv-x-playground/RWKV-v5
TRAINER_DIR: /root/rwkv-x-playground/RWKV-v5
PROJECT_DIR: /root/rwkv-x-playground


## Stage 1 : Foundation 4k model training

In [3]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/V5-Base-1B5-enwiki-4k.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 80.96it/s]
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-f7b060a194034cc1_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-5130e0b5cfea4510_*_of_00064.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-8cf337c8a178675e_*_of_00064.arrow
Loading cached split indices for

In [4]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/V5-Base-1B5-enwiki-4k.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-4k Foundation (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=1

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/V5-Base-1B5/V5-Base-1B5-enwiki-4k.yaml', '--trainer.logger.init_args.name=V5-Base-1B5 - Enwiki-4k Foundation (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--model.ctx_len=4096', '--model.bptt_learning_range=1'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/V5-Base-1B5/V5-Base-1B5-enwiki-4k.yaml', '--trainer.logger.init_args.name=V5-Base-1B5 - Enwiki-4k Foundation (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=de

In [5]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/V5-Base-1B5-enwiki-4k/last.ckpt" "../model/V5-Base-1B5-Enwiki-4k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/V5-Base-1B5-Enwiki-4k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/V5-Base-1B5-enwiki-4k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 4
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/V5-Base-1B5-Enwiki-4k.pth
-rw-r--r-- 1 root root 5.7G Aug 12 19:28 ../model/V5-Base-1B5-Enwiki-4k.pth


In [6]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py "../model/V5-Base-1B5-Enwiki-4k.pth" "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. They named the animal as the son of a gigantic fox and a big deer. This coyote was called "the African tiger" because of its similarity to the Alpine hive, the Himalayan hive, and the wild horses of the region. The buffalo also owned a large mammal, called the Siberian Horse, and a mongo (Wakian jub) named for a leopard. The animal also gained a reputation for its hot summers and other wild dogs, especially when they ran aground on their way to attack a bear in China.

In April 2017, the fourth American black bear was discovered in the Pacific, and there is no evidence that it was found in the Wyoming. The discovery is not clear. In a 

In [7]:
# # Lets do a quick memory test
# # (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
# !python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/V5-Base-1B5-Stage1.pth"

## Stage 2 : Foundation 16k model training

In [8]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/V5-Base-1B5-enwiki-16k.yaml"

Found cached dataset parquet (/root/.cache/huggingface/datasets/teven___parquet/teven--enwiki_100k-1359e81b212c2dd6/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.96it/s]


In [9]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/V5-Base-1B5-enwiki-16k.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Enwiki-16k Foundation (train-ctx=4k, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.ctx_len=4096 \
        --model.bptt_learning_range=4

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/V5-Base-1B5/V5-Base-1B5-enwiki-16k.yaml', '--trainer.logger.init_args.name=V5-Base-1B5 - Enwiki-16k Foundation (train-ctx=4k, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--model.ctx_len=4096', '--model.bptt_learning_range=4'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/V5-Base-1B5/V5-Base-1B5-enwiki-16k.yaml', '--trainer.logger.init_args.name=V5-Base-1B5 - Enwiki-16k Foundation (train-ctx=4k, deepspeed_stage_1)', '--trainer.strateg

In [10]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/V5-Base-1B5-enwiki-16k/last.ckpt" "../model/V5-Base-1B5-Enwiki-16k.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/V5-Base-1B5-Enwiki-16k.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/V5-Base-1B5-enwiki-16k/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 4
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/V5-Base-1B5-Enwiki-16k.pth
-rw-r--r-- 1 root root 5.7G Aug 13 06:02 ../model/V5-Base-1B5-Enwiki-16k.pth


In [11]:
# # Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py "../model/V5-Base-1B5-Enwiki-16k.pth" "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. The researchers were not interested in the game, as they thought the creatures are too advanced and far too easy.

At the time, most scientists thought that the human language was in the language, but that they had to travel in other directions. Many scientists, such as Leopold Stolz, believed that the dragon was a real human language, and that the people there saw a more "fantasy" appearance. Some scientists doubted that the human language had a human-like origin, and thought it was more useful to show that a human is a human language.

The scientists also suggested that the dragon language was a spiritual or psychological-mechanistic

In [12]:
# # Lets do a quick memory test
# # (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
# !python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/V5-Base-1B5-Stage1.pth"

# Stage 3 : Instruct Tuning

In [13]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/V5-Base-1B5-instruct.yaml"

Extracting data files: 100%|████████████████████| 1/1 [00:00<00:00, 2267.19it/s]
Setting num_proc from 64 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/c-s-ale___parquet/c-s-ale--dolly-15k-instruction-alpaca-format-9dfbb23260d63d9d/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 523.11it/s]


In [14]:
# Start the instruct finetuning
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/V5-Base-1B5-instruct.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Instruct (train-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/V5-Base-1B5/V5-Base-1B5-instruct.yaml', '--trainer.logger.init_args.name=V5-Base-1B5 - Instruct (train-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'], args=['fit', '-c', '/root/rwkv-x-playground/notebook/experiment/rwkv-x-exp/V5-Base-1B5/V5-Base-1B5-instruct.yaml', '--trainer.logger.init_args.name=V5-Base-1B5 - Instruct (train-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'].
  rank_zero_warn(
/usr/local/lib/p

In [15]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/V5-Base-1B5-instruct/last.ckpt" "../model/V5-Base-1B5-Enwiki-Instruct.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/V5-Base-1B5-Enwiki-Instruct.pth"

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/V5-Base-1B5-instruct/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 4
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 486 params 1515107840 elements
Saving fp32 state dict to ../model/V5-Base-1B5-Enwiki-Instruct.pth
-rw-r--r-- 1 root root 5.7G Aug 13 06:39 ../model/V5-Base-1B5-Enwiki-Instruct.pth


In [16]:
# Lets do a quick dragon prompt validation
!cd "{INFERENCE_DIR}" && python3 dragon_test.py "../model/V5-Base-1B5-Enwiki-Instruct.pth" "cuda fp32"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
--- DRAGON PROMPT ---
In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese. In a vivid, energetic and convincing story, the dragon brought his birth to earth.

The dragon, finding that dragons, reached his destination, eventually conquered the dragon, and transformed the dragon into the dragon.
The dragon is the second-oldest of the dragon, its dragon, but the dragon did not rest in his path.
The dragon of its dragon, Mian Zimura, the first dragon in the modern world.
The dragon was intended to move the dragon in the future.
The dragon first appeared in the dragon's own mind.
The dragon's dragon had magical and immobile as a human, one reason it was a waste, to also help to bring any dragon in his quest.

As o

In [17]:
# # Lets do a quick memory test
# # (We dun expect this to work, as we have not finetune for memory recall, but its a baseline)
# !python3 ../memory_script/eval_v5_memory_guided.py "{PROJECT_DIR}/model/V5-Base-1B5-Enwiki-Instruct.pth"